In [ ]:
# Code to calculate percentages of soil type / landuse cathegory in each basin. 
# Soil type or landuse cathegory is ment to be stored in tif file.
# Catchment should be in vectors (geopackage or shapefile).
# Code is counting with borders, where percentage of cell inside the polygon is calculated.
# Created by Vit Kovacka on 2023-12-20
# CAFL Aon Impact Forecasting project

# This Jupyter version is not running paralelly. 

In [ ]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
import os

In [ ]:
# Load the raster data
raster_file = r"V:\Canada\Soil\Soil_perc_testing\testing_raster_Fraser.tif"
raster = rasterio.open(raster_file)
print(raster.crs)
with rasterio.open(raster_file) as src:
    image = src.read(1)  # read the first band
    results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) in enumerate(shapes(image, transform=src.transform))
    )

# Load the polygon data
polygon_file = r"V:\Canada\Soil\Soil_perc_testing\testing_catch_Fraser.gpkg"
polygon_gdf = gpd.read_file(polygon_file)
print(polygon_gdf.crs)


# Define the directory where the CSV files will be saved
output_dir = r"V:\Canada\Soil\Soil_perc_testing\output"
# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Convert the results to a GeoDataFrame
raster_gdf = gpd.GeoDataFrame.from_features(list(results))
print("Raster conversion done.")

# Get the total number of polygons
total_polygons = len(polygon_gdf)

# Assuming 'polygon_gdf' is a GeoDataFrame with multiple polygons
for i, (idx, row) in enumerate(polygon_gdf.iterrows()):
    single_polygon = gpd.GeoDataFrame([row])

    # Calculate the intersection of the raster polygons and the single polygon
    intersection = gpd.overlay(raster_gdf, single_polygon, how='intersection')

    # Calculate the area of each intersection
    intersection['area'] = intersection.geometry.area

    # Calculate the total area covered by each unique raster value
    total_areas = intersection.groupby('raster_val')['area'].sum()

    # Calculate the overall percentages
    total_area = total_areas.sum()
    overall_percentages = (total_areas / total_area) * 100

    # Print the current iteration number and the total number of polygons
    print(f"Calculating for polygon {i+1} out of {total_polygons}")
    
    # Print the polygon COMID and overall percentages
    print(f"Polygon COMID: {row['COMID']}")
    print(overall_percentages)

    # Save the percentages to a CSV file
    output_file = os.path.join(output_dir, f"{row['COMID']}_percentages.csv")
    overall_percentages.to_csv(output_file, header=True)